**Dealing with a large dataset on your computer** is a common challenge for data analysts. If you need to process or filter the data you can set the `chunksize=` argument of pandas `read_csv()` method to loop through and work with manageable chunks of the data.

If for example, you wanted to work with a large data file (HCPS_data.csv) to pull just the rows where the `HCPS Code` is **99213**, you could read that file to a chunk of 100,000 rows at a time, filter each chunk to the rows with the specified code, save the filtered results of each chunk to a list and concatenate them all together at the end. The syntax would look something like this:

```
code_99213_rows =[]

for chunk in pd.read_csv('HCPS_data.csv', chunksize = 100000):
    code_99213_rows.append(chunk[chunk['HCPS Code'] == '99213']) 
               
code_99213_df = pd.concat(code_99213_rows, ignore_index=True)
```
======================================================================   

To shrink the size of a file so that it loads more quickly, converting a text file (CSV) to binary might make sense. In python, you can work with data to minimize its footprint and then store the resulting object (dataframe) as a [pickle](https://docs.python.org/3/library/pickle.html) file.

In [2]:
import pandas as pd
import pickle

In [13]:
%%time
may = pd.read_csv('../data/may.csv')
may.head()
june = pd.read_csv('../data/june.csv')
june.head()
july = pd.read_csv('../data/july.csv')
july.head()

Wall time: 2min 30s


,pubdatetime,latitude,longitude,sumdid,sumdtype,chargelevel,sumdgroup,costpermin,companyname
0,2019-07-01 00:00:33.550000,36.156678,-86.809004,Powered635135,Powered,22.0,scooter,0.15,Lyft
1,2019-07-01 00:00:34.973000,36.145674,-86.794138,Powered790946,Powered,33.0,scooter,0.15,Lyft
2,2019-07-01 00:00:41.183000,36.179319,-86.751538,Powered570380,Powered,76.0,scooter,0.15,Lyft
3,2019-07-01 00:00:41.620000,36.152111,-86.803821,Powered240631,Powered,43.0,scooter,0.15,Lyft
4,2019-07-01 00:00:45.087000,36.149355,-86.797550,Powered970404,Powered,52.0,scooter,0.15,Lyft


### Now try to reduce the size of the file
- objects require the most space


In [6]:
may.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20292503 entries, 0 to 20292502
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   pubdatetime  object 
 1   latitude     float64
 2   longitude    float64
 3   sumdid       object 
 4   sumdtype     object 
 5   chargelevel  float64
 6   sumdgroup    object 
 7   costpermin   float64
 8   companyname  object 
dtypes: float64(4), object(5)
memory usage: 1.4+ GB


#### convert the company name to an integer 
- find the unique company names
- assign each company an integer (you can use a dictionary for this step)
- update the `companyname` column to store the integer id for each company

In [16]:
may.companyname.unique()

array(['Bird', 'Lyft', 'Gotcha', 'Lime', 'Spin', 'Jump', 'Bolt'],
      dtype=object)

In [15]:
june.companyname.unique()

array(['Jump', 'Bird', 'Bolt', 'Gotcha', 'Spin', 'Lime', 'Lyft'],
      dtype=object)

In [17]:
july.companyname.unique()

array(['Lyft', 'Bird', 'Spin', 'Bolt', 'Jump', 'Lime', 'Gotcha'],
      dtype=object)

In [18]:
company_dict = {'Bird':0, 'Lyft': 1, 'Gotcha': 2, 'Lime': 3, 'Spin': 4, 'Jump': 5, 'Bolt': 6}

In [19]:
may.companyname = may.companyname.replace(company_dict)
june.companyname = june.companyname.replace(company_dict)
july.companyname = july.companyname.replace(company_dict)

#### next convert `pubdatetime` to a datetime 

In [20]:
may.pubdatetime = pd.to_datetime(may.pubdatetime)
june.pubdatetime = pd.to_datetime(june.pubdatetime)
july.pubdatetime = pd.to_datetime(july.pubdatetime)

#### Next remove unneeded data
#### keep just the scooters

In [21]:
may.sumdgroup.unique()
june.sumdgroup.unique()
july.sumdgroup.unique()

array(['scooter', 'Scooter', 'bicycle'], dtype=object)

In [22]:
may_scooters = may.loc[may.sumdgroup.isin(['scooter', 'Scooter'])]
june_scooters = june.loc[june.sumdgroup.isin(['scooter', 'Scooter'])]
july_scooters = july.loc[july.sumdgroup.isin(['scooter', 'Scooter'])]

#### keep just the columns we want to work with

In [23]:
may_scooters = may_scooters[['pubdatetime', 'latitude', 'longitude', 'sumdid', 'chargelevel', 'companyname']]
june_scooters = june_scooters[['pubdatetime', 'latitude', 'longitude', 'sumdid', 'chargelevel', 'companyname']]
july_scooters = july_scooters[['pubdatetime', 'latitude', 'longitude', 'sumdid', 'chargelevel', 'companyname']]

#### check `.info()` again

In [24]:
may_scooters.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20283582 entries, 0 to 20292502
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   pubdatetime  datetime64[ns]
 1   latitude     float64       
 2   longitude    float64       
 3   sumdid       object        
 4   chargelevel  float64       
 5   companyname  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 1.1+ GB


#### The only object datatype remaining is sumdid (an alphanumeric unique identifier)
- time to pickle

In [26]:
may_scooters.to_pickle("../data/may.pkl")
june_scooters.to_pickle("../data/june.pkl")
july_scooters.to_pickle("../data/july.pkl")

In [29]:
%%time
may_test = pd.read_pickle("../data/may.pkl")
june_test = pd.read_pickle("../data/june.pkl")
july_test = pd.read_pickle("../data/july.pkl")

Wall time: 18.8 s


As of 10/12/2021 at 8:30pm CST, this ran successfully bringing the file read time from 2m 30s to 18.8s.

Read in the trip files now.

In [3]:
%%time
may_trip = pd.read_csv('../data/may_trip.csv')
june_trip = pd.read_csv('../data/june_trip.csv')
july_trip = pd.read_csv('../data/july_trip.csv')
july_trip.head()

Wall time: 16.2 s


,pubTimeStamp,companyName,tripRecordNum,sumdID,tripDuration,tripDistance,startDate,startTime,endDate,endTime,startLatitude,startLongitude,endLatitude,endLongitude,tripRoute,create_dt
0,2019-07-01 00:02:52.213000,Bird,BRD5179,PoweredZSHKJ,2.0,0.0,2019-07-01 00:00:00,00:01:32.290000,2019-07-01 00:00:00,00:03:55.966666,36.1775,-86.7516,36.1778,-86.7514,"[[36.177438,-86.751861],[36.177425,-86.751987]...",2019-07-02 05:30:18.333000
1,2019-07-01 00:02:52.213000,Bird,BRD5177,Powered22JF6,0.0,0.0,2019-07-01 00:00:00,00:02:38.640000,2019-07-01 00:00:00,00:02:54.406666,36.1627,-86.7711,36.1627,-86.7711,"[[36.162718,-86.771102],[36.162718,-86.771102]]",2019-07-02 05:30:18.273000
2,2019-07-01 00:02:52.213000,Bird,BRD5176,Powered22JF6,1.0,0.0,2019-07-01 00:00:00,00:03:04.333333,2019-07-01 00:00:00,00:03:37.680000,36.1627,-86.7713,36.1627,-86.7713,"[[36.162734,-86.77122],[36.162706,-86.771325],...",2019-07-02 05:30:18.243000
3,2019-07-01 00:02:52.213000,Bird,BRD5178,PoweredIB2CC,1.0,0.0,2019-07-01 00:00:00,00:02:06.963333,2019-07-01 00:00:00,00:03:25.766666,36.1525,-86.7988,36.1525,-86.7988,"[[36.152499,-86.798708]]",2019-07-02 05:30:18.303000
4,2019-07-01 00:02:52.213000,Bird,BRD5180,PoweredW4G7R,2.0,0.0,2019-07-01 00:00:00,00:01:19.843333,2019-07-01 00:00:00,00:02:53.976666,36.1578,-86.7760,36.1575,-86.7758,"[[36.157523,-86.775794]]",2019-07-02 05:30:18.363000


Replace the company names with an integer

In [6]:
company_dict = {'Bird':0, 'Lyft': 1, 'Gotcha': 2, 'Lime': 3, 'Spin': 4, 'Jump': 5, 'Bolt': 6}
may_trip.companyName = may_trip.companyName.replace(company_dict)
june_trip.companyName = june_trip.companyName.replace(company_dict)
july_trip.companyName = july_trip.companyName.replace(company_dict)

Converting pubTimeStamp to datetime

In [7]:
may_trip.pubTimeStamp = pd.to_datetime(may_trip.pubTimeStamp)
june_trip.pubTimeStamp = pd.to_datetime(june_trip.pubTimeStamp)
july_trip.pubTimeStamp = pd.to_datetime(july_trip.pubTimeStamp)

See if there is anything else we need to know about the data

In [10]:
may_trip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224969 entries, 0 to 224968
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   pubTimeStamp    224969 non-null  datetime64[ns]
 1   companyName     224969 non-null  object        
 2   tripRecordNum   224969 non-null  object        
 3   sumdID          224969 non-null  object        
 4   tripDuration    224969 non-null  float64       
 5   tripDistance    224969 non-null  float64       
 6   startDate       224969 non-null  object        
 7   startTime       224969 non-null  object        
 8   endDate         224969 non-null  object        
 9   endTime         224969 non-null  object        
 10  startLatitude   224969 non-null  float64       
 11  startLongitude  224969 non-null  float64       
 12  endLatitude     224969 non-null  float64       
 13  endLongitude    224969 non-null  float64       
 14  tripRoute       224969 non-null  obj

Convert startDate, startTime, endDate, and endTime to times and dates

In [ ]:
may_trip.startDate = pd.to_datetime(may_trip.startDate)
may_trip.startTime = pd.to_datetime(may_trip.startTime)
may_trip.endDate = pd.to_datetime(may_trip.endDate)
may_trip.endTime = pd.to_datetime(may_trip.endTime)

Remove everything except for the columns we need

In [9]:
may_trip_scooters = may_scooters[['pubdatetime', 'latitude', 'longitude', 'sumdid', 'chargelevel', 'companyname']]
june_trip_scooters = june_scooters[['pubdatetime', 'latitude', 'longitude', 'sumdid', 'chargelevel', 'companyname']]
july_trip_scooters = july_scooters[['pubdatetime', 'latitude', 'longitude', 'sumdid', 'chargelevel', 'companyname']]

NameError: name 'may_trip_scooters' is not defined